
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [118]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

pd.set_option('display.max_columns', 500)

In [134]:
# ChatGPT 
def get_nested_value(data, *keys):
    try:
        for key in keys:
            data = data[key]
        return data
    except (TypeError, KeyError):
        return None

#make list of file names in data folder
file_list = os.listdir('../data')

# Create a dataframe to store all data
all_data = pd.DataFrame()

i = 0

#loop through filenames in list 
for file in file_list:
    i += 1
    print(i)
    try:
        #load json object
        with open('../data/' + file) as f:
            nested_json = json.load(f)
            response_data = nested_json['data']['results']
            # List to store data from file
            extracted_data = []
            for result in response_data:
                row = {
                    'branding_name': nested_json['data']['results'][0]['branding'][0]['name'],  
                    'baths': get_nested_value(result, 'description', 'baths'),
                    'baths_1qtr': get_nested_value(result, 'description', 'baths_1qtr'),
                    'baths_3qr': get_nested_value(result, 'description', 'baths_3qr'),
                    'baths_full': get_nested_value(result, 'description', 'baths_full'),
                    'baths_half': get_nested_value(result, 'description', 'baths_half'),                          
                    # Still need to add ('beds','garage','lot_sqft','name','sold_date','sold_price', 'sqft', 'stories', 'sub_type', 'type', 'year_built')
                    
                    'is_coming_soon': get_nested_value(result, 'flags', 'is_coming_soon'),
                    'is_contingent': get_nested_value(result, 'flags', 'is_contingent'),
                    # Still need to add ('is_for_rent', 'is_foreclosure', 'is_new_construction', 'is_new_listing', 'is_pending', 'is_plan', 'is_price_reduced', 'is_subdivision')
                    
                    'last_update_date': get_nested_value(result, 'last_update_date'),
                    'show_contact_an_agent': get_nested_value(result, 'lead_attributes', 'show_contact_an_agent'),
                    'list_date': get_nested_value(result, 'list_date'),
                    'list_price': get_nested_value(result, 'list_price'),
                    'listing_id': get_nested_value(result, 'listing_id'),
                    'city': get_nested_value(result, 'location', 'address', 'city'),
                    'lat': get_nested_value(result, 'location', 'address', 'coordinate', 'lat'),
                    'lon': get_nested_value(result, 'location', 'address', 'coordinate', 'lon'),
                    'line': get_nested_value(result, 'location', 'address', 'line'),
                    'postal_code': get_nested_value(result, 'location', 'address', 'postal_code'),
                    'state': get_nested_value(result, 'location', 'address', 'state'),
                    'state_code': get_nested_value(result, 'location', 'address', 'state_code'),
                    'open_houses': get_nested_value(result, 'open_houses'),
                    'price_reduced_amount': get_nested_value(result, 'price_reduced_amount'),
                    'brand_name': get_nested_value(result, 'products', 'brand_name'),
                    'property_id': get_nested_value(result, 'property_id'),
                    'status': get_nested_value(result, 'status'),
                    'tags': get_nested_value(result, 'tags'),
                }
                extracted_data.append(row)
        extracted_dataframe = pd.DataFrame(extracted_data).drop_duplicates("property_id")
        all_data = pd.concat([all_data, extracted_dataframe])
    except:
        continue

print(all_data.info())


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
<class 'pandas.core.frame.DataFrame'>
Index: 8159 entries, 0 to 41
Data columns (total 27 columns):
 #  

In [150]:
all_data.iloc[:, :-1].drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 0 to 41
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   branding_name          6958 non-null   object 
 1   baths                  6915 non-null   float64
 2   baths_1qtr             0 non-null      object 
 3   baths_3qr              0 non-null      object 
 4   baths_full             6414 non-null   object 
 5   baths_half             1986 non-null   object 
 6   is_coming_soon         0 non-null      object 
 7   is_contingent          0 non-null      object 
 8   last_update_date       7038 non-null   object 
 9   show_contact_an_agent  7065 non-null   bool   
 10  list_date              6765 non-null   object 
 11  list_price             6739 non-null   object 
 12  listing_id             6765 non-null   object 
 13  city                   7063 non-null   object 
 14  lat                    6879 non-null   object 
 15  lon        

In [121]:
string_data = all_data

string_data.loc[:, 'tags'] = string_data['tags'].apply(lambda x: tags_to_string(x))

In [122]:
string_data.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 0 to 41
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   branding_name          42 non-null     object 
 1   community              0 non-null      object 
 2   baths                  42 non-null     int64  
 3   baths_1qtr             0 non-null      object 
 4   baths_3qr              0 non-null      object 
 5   baths_full             39 non-null     float64
 6   baths_half             9 non-null      float64
 7   is_coming_soon         0 non-null      object 
 8   is_contingent          0 non-null      object 
 9   last_update_date       42 non-null     object 
 10  show_contact_an_agent  42 non-null     bool   
 11  list_date              42 non-null     object 
 12  list_price             42 non-null     int64  
 13  listing_id             42 non-null     object 
 14  city                   42 non-null     object 
 15  lat          

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [123]:
# load and concatenate data here
# drop or replace values as necessary

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [124]:
# OHE categorical variables here
# tags will have to be done manually

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [125]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [126]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [127]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized